In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [2]:
url='https://chipublib.bibliocommons.com/events/rss/all'
program_list={}
my_dict = {"program_name":[],
           "program_description":[],
           "program_url":[],
           "virtual":[],
           "registration_url":[],
           "start_date":[],
           "end_date":[],
           "program":[],
           "category":[],
          "audience":[],
          "language":[],
          "location_name":[],
          "address":[],
          "city":[],
          "state":[],
          "zipcode":[],
          "contact_name":[],
          "contact_phone":[],
          "contact_email":[]}

In [3]:
r=requests.get(url)
soup=bs(r.text, features='xml')
print('CPL scrape successful: ',r.status_code)
programs=soup.find_all('item')
len(programs)

CPL scrape successful:  200


1297

In [4]:
for p in programs:
            title=p.find('title').text
            description=p.find('description').text.strip()
            link=p.find('link').text
            virtual=p.find('bc:is_virtual').text
            start_date=p.find('bc:start_date_local').text
            end_date=p.find('bc:end_date_local').text
            program=getattr(p.find('category',domain='Program'), 'text', 'null')
            category=getattr(p.find('category',domain='Type'), 'text', 'null')
            audience=p.find_all('category',domain='Audience')
            language=p.find_all('category',domain='Language')
            location=p.find_all('bc:location')
            if location is not None:
                for l in location:
                    location_name=getattr(l.find('bc:name'), 'string', None)
                
            location_number=getattr(p.find('bc:number'), 'string', 'null')
            location_street=getattr(p.find('bc:street'), 'string', 'null')
            if location_number!=location_street:
                location_address=f'{location_number} {location_street}'
            else:
                location_address=location_number
            
            location_city=getattr(p.find('bc:city'), 'string', 'null')
            location_zip=getattr(p.find('bc:zip'), 'string', 'null')
            location_state=getattr(p.find('bc:state'), 'string', 'null')
            contact_name=p.find('bc:contact')
            contact_phone=getattr(p.find('bc:phone'), 'string', 'null')
            contact_email=getattr(p.find('bc:email'), 'string', 'null')
            
            
            my_dict["program_name"].append(title)
            my_dict["program_description"].append(description)
            my_dict["program_url"].append(link)
            my_dict["registration_url"].append(link)
            my_dict["virtual"].append(virtual)
            my_dict["start_date"].append(start_date)
            my_dict["end_date"].append(start_date)
            my_dict["program"].append(program)
            my_dict["category"].append(category)
            my_dict["audience"].append(audience)
            my_dict["language"].append(language)
            my_dict["location_name"].append(location_name)
            my_dict["address"].append(location_address)
            my_dict["city"].append(location_city)
            my_dict["state"].append(location_state)
            my_dict["zipcode"].append(location_zip)
            my_dict["contact_name"].append(contact_name)
            my_dict["contact_phone"].append(contact_phone)
            my_dict["contact_email"].append(contact_email)
            
#             program={'title':title,
#                     'link':link,
#                     'start date':start_date
#                     }
#             program_list.append(program)

In [5]:
df = pd.DataFrame(my_dict)
df.head(30)

,program_name,program_description,program_url,virtual,registration_url,start_date,end_date,program,category,audience,language,location_name,address,city,state,zipcode,contact_name,contact_phone,contact_email
0,Grab and Go: Paper Rose Craft,<p>In celebration of Latinx History Month come...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-08T09:00,2021-10-08T09:00,Latinx History,"Crafts, Games and Play",[[Kids]],[[English]],Manning,6 S. Hoyne Avenue,Chicago,IL,60612,"[[Ms. Jade], [+1-312-746-6800], [jnewman@chipu...",+1-312-746-6800,jnewman@chipublib.org
1,Rise and Shine Story Time: The Alphabet,<p>Start your day the library way!&nbsp;Join M...,https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2021-10-08T09:30,2021-10-08T09:30,null,Story Time,"[[Babies], [Preschoolers], [Toddlers]]",[[English]],Manning,null,null,null,null,"[[Jerry McNally], [+1-312-744-1965]]",+1-312-744-1965,null
2,Family Story Time,<p>Join the Thomas Hughes Children's Library s...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-08T10:00,2021-10-08T10:00,null,Story Time,"[[Babies], [Preschoolers], [Toddlers]]",[[English]],Manning,null,null,null,null,"[[Thomas Hughes Children's Library], [+1-312-7...",+1-312-747-4200,null
3,Creative Writing Workshop,"<p>Turn over a new leaf, this autumn, and cele...",https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-08T10:00,2021-10-08T10:00,null,Writing and Poetry,[[Adults]],[[English]],Manning,null,null,null,null,"[[Tom Stark], [+1-312-742-9590], [tstark@chipu...",+1-312-742-9590,tstark@chipublib.org
4,Story Time with Ms. Katie,<p>Join Ms.&nbsp;Katie from the McKinley Park ...,https://chipublib.bibliocommons.com/events/610...,true,https://chipublib.bibliocommons.com/events/610...,2021-10-08T10:30,2021-10-08T10:30,null,Story Time,"[[Babies], [Preschoolers], [Toddlers]]",[[English]],Manning,null,null,null,null,"[[Ms. Katie], [+1-312-747-6082], [kboucher@chi...",+1-312-747-6082,kboucher@chipublib.org
5,Preschool Story Time,<p>Acoustic guitar story times at the Logan Sq...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-08T10:30,2021-10-08T10:30,null,Story Time,[[Preschoolers]],[[English]],Logan Square,3030 W. Fullerton Avenue,Chicago,IL,60647,"[[Ms. Erica], [+1-312-744-5295], [egamble@chip...",+1-312-744-5295,egamble@chipublib.org
6,Latin American Folktales Story Time,"<figure><img src=""https://chipublib.bibliocomm...",https://chipublib.bibliocommons.com/events/610...,true,https://chipublib.bibliocommons.com/events/610...,2021-10-08T10:30,2021-10-08T10:30,Latinx History,Story Time,"[[Babies], [Kids], [Preschoolers], [Toddlers]]",[[English]],Logan Square,null,null,null,null,"[[Lily], [+1-312-744-0900]]",+1-312-744-0900,null
7,Strategies for Claiming Social Security Benefits,<p>Starting your Social Security benefits may ...,https://chipublib.bibliocommons.com/events/614...,true,https://chipublib.bibliocommons.com/events/614...,2021-10-08T12:00,2021-10-08T12:00,null,"Business, Law and Money",[[Adults]],[[English]],Logan Square,null,null,null,null,"[[Chloe], [+1-312-746-6800]]",+1-312-746-6800,null
8,Adult Chit Chat: Queen Sugar,<p>Join us for a&nbsp;casual chit-chat on&nbsp...,https://chipublib.bibliocommons.com/events/614...,true,https://chipublib.bibliocommons.com/events/614...,2021-10-08T13:00,2021-10-08T13:00,null,Book Clubs,[[Adults]],[[English]],Logan Square,null,null,null,null,"[[Cydni Hinton], [+1-312-745-1608], [chinton@c...",+1-312-745-1608,chinton@chipublib.org
9,After School Meals,<p>Free after school meals are provided by the...,https://chipublib.bibliocommons.com/events/614...,false,https://chipublib.bibliocommons.com/events/614...,2021-10-08T15:30,2021-10-08T15:30,null,Health and Science,"[[Kids], [Preschoole

In [6]:
df=df.astype({"language":'string',
             "audience":'string',
             "contact_name":'string'})

In [7]:
df['location_name'] = np.where(df["virtual"]=='true', 'null', df['location_name'])
df['meeting_type'] = np.where(df["virtual"]=='true', 'online', 'face_to_face')
#df['price']=np.where(df["virtual"] is not None, 0, 0)
df['price']=0
df[['start_date','start_time']] = df['start_date'].str.split('T',expand=True)
df[['end_date','end_time']] = df['end_date'].str.split('T',expand=True)

In [8]:
df["contact_name"]=df["contact_name"].str.split(']',1,expand=True)
df['contact_name'] = df['contact_name'].str.replace(r'<[^<>]*>', '', regex=True)
df['contact_name'] = df['contact_name'].str.replace('[','')
df['contact_name'] = df['contact_name'].str.replace(']','')
df["contact_name"]=df["contact_name"].str.split('+',1,expand=True)

In [9]:
df['contact_phone'] = df['contact_phone'].str.replace('+','')
df['contact_phone'] = df['contact_phone'].str.replace('1-','')
df['contact_phone'] = df['contact_phone'].str.replace('-','')

In [10]:
df['internal_id']=df['program_url']
df["internal_id"]=df["internal_id"].str.strip('https://chipublib.bibliocommons.com/events/')
df["internal_id"]

0       61571f6f66359037007e6f8f
1       615b087bddad1a3000c2254f
2       612e35a6577350360031868f
3       612fc678cf94553600290f05
4       610d3daf8b28013800490fd8
                  ...           
1292     61520689d68b0a2f003c329
1293    61520689d68b0a2f003c329d
1294     61520689d68b0a2f003c329
1295    611ab2eaad0dd42800ed2751
1296     60e46918080ccb420009628
Name: internal_id, Length: 1297, dtype: object

In [11]:
df

,program_name,program_description,program_url,virtual,registration_url,start_date,end_date,program,category,audience,...,state,zipcode,contact_name,contact_phone,contact_email,meeting_type,price,start_time,end_time,internal_id
0,Grab and Go: Paper Rose Craft,<p>In celebration of Latinx History Month come...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-08,2021-10-08,Latinx History,"Crafts, Games and Play","[<category domain=""Audience"">Kids</category>]",...,IL,60612,Ms. Jade,3127466800,jnewman@chipublib.org,face_to_face,0,09:00,09:00,61571f6f66359037007e6f8f
1,Rise and Shine Story Time: The Alphabet,<p>Start your day the library way!&nbsp;Join M...,https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2021-10-08,2021-10-08,null,Story Time,"[<category domain=""Audience"">Babies</category>...",...,null,null,Jerry McNally,3127441965,null,online,0,09:30,09:30,615b087bddad1a3000c2254f
2,Family Story Time,<p>Join the Thomas Hughes Children's Library s...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-08,2021-10-08,null,Story Time,"[<category domain=""Audience"">Babies</category>...",...,null,null,Thomas Hughes Children's Library,3127474200,null,online,0,10:00,10:00,612e35a6577350360031868f
3,Creative Writing Workshop,"<p>Turn over a new leaf, this autumn, and cele...",https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-08,2021-10-08,null,Writing and Poetry,"[<category domain=""Audience"">Adults</category>]",...,null,null,Tom Stark,3127429590,tstark@chipublib.org,online,0,10:00,10:00,612fc678cf94553600290f05
4,Story Time with Ms. Katie,<p>Join Ms.&nbsp;Katie from the McKinley Park ...,https://chipublib.bibliocommons.com/events/610...,true,https://chipublib.bibliocommons.com/events/610...,2021-10-08,2021-10-08,null,Story Time,"[<category domain=""Audience"">Babies</category>...",...,null,null,Ms. Katie,3127476082,kboucher@chipublib.org,online,0,10:30,10:30,610d3daf8b28013800490fd8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292,Books 'n Babes,"<p>Come one, come all, to our fun story time f...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-02-28,2022-02-28,null,Story Time,"[<category domain=""Audience"">Babies</category>]",...,null,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,0,10:30,10:30,61520689d68b0a2f003c329
1293,Books 'n Babes,"<p>Come one, come all, to our fun story time f...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-03-14,2022-03-14,null,Story Time,"[<category domain=""Audience"">Babies</category>]",...,null,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,0,10:30,10:30,61520689d68b0a2f003c329d
1294,Books 'n Babes,"<p>Come one, come all, to our fun story time f...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-03-28,2022-03-28,null,Story Time,"[<category domain=""Audience"">Babies</category>]",...,null,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,0,10:30,10:30,61520689d68b0a2f003c329
1295,"Law at the Library: Wills, Trusts and Estate P...","<figure><img src=""https://chipublib.bibliocomm...",https://chipublib.bibliocommons.com/events/611...,true,https://chipublib.bibliocommons.com/events/611...,2022-03-29,2022-03-29,Law at the Library,"Business, Law and Money","[<category domain=""Audience"">Adults</category>]",...,null,null,adultservices@chipublib.org,null,adultservices@chipublib.org,online,0,18:00,18:00,611ab2eaad0dd42800ed2751


In [12]:
# def cpl_programs(url):
#     program_list=[]
#     try:
#         r=requests.get(url)
#         soup=BeautifulSoup(r.content, features='xml')
# #         return print('CPL scrape successful: ',r.status_code)
#         programs = soup.findAll('item')
    
#         for p in programs:
#             title=p.find('title').text
#             link=p.find('link').text
#             start_date=p.find('bc:start_date_local').text
            
#             program={'title':title,
#                     'link':link,
#                     'start date':start_date
#                     }
#             program_list.append(program)
        

#         return print(program_list)
#     except Exception as e:
#         print('FAILED. See exception: ')
#         print(e)
        


In [13]:
# print('Starting scrape')
# cpl_programs(url)
# print('Finished')

In [14]:
cat_list=df['category'].unique()

In [15]:
cat_list

array(['Crafts, Games and Play', 'Story Time', 'Writing and Poetry',
       'Business, Law and Money', 'Book Clubs', 'Health and Science',
       'DIY', 'Art, Movies and Performances', 'Computers and Technology',
       'Tours', 'Workshops', 'Continuing Education and College',
       'History and Genealogy', 'Jobs and Careers', 'Celebrations',
       'Lectures', 'Author Events', 'Library Closures'], dtype=object)

In [16]:
len(cat_list)

18

In [17]:
#MCMF_list=list(range(1,len(cat_list)+1))

In [18]:
MCMF_list=[232,230,215,223,232,222,230,228,218,232,232,223,230,224,210,215,221,230,215]

In [19]:
MCMF_list=list(map(str, MCMF_list))

In [20]:
# ids = {'lib_cats':cat_list,
#       'MCMF_cat':MCMF_list}

# ids = dict(zip(ids['lib_cats'], ids['MCMF_cat']))

In [21]:
cat_dict={'Crafts, Games and Play':'232',
         'Story Time':'230',
         'Writing and Poetry':'230',
         'Business, Law and Money':'224',
         'Book Clubs':'230',
         'Health and Science':'218',
         'DIY':'210',
         'Art, Movies and Performances':'232',
         'Computers and Technology':'228',
         'Tours':'215',
         'Workshops':'232',
         'Continuing Education and College':'223',
         'History and Genealogy':'221',
         'Jobs and Careers':'222',
         'Celebrations':'232',
         'Lectures':'223',
         'Author Events':'230',
         'Library Closures':'215'}

In [22]:
for old, new in cat_dict.items():
    df['category'] = df['category'].str.replace(old, new, regex=False)

In [23]:
df['language'] = df['language'].str.replace(r'<[^<>]*>', '', regex=True)
df['language'] = df['language'].str.replace('[','')
df['language'] = df['language'].str.replace(']','')

In [24]:
df['audience'] = df['audience'].str.replace(r'<[^<>]*>', '', regex=True)
df['audience'] = df['audience'].str.replace('[','')
df['audience'] = df['audience'].str.replace(']','')
df['audience_max']=df['audience']

In [25]:
df['program_description'] = df['program_description'].str.replace(r'<[^<>]*>', '', regex=True)

In [26]:
df

,program_name,program_description,program_url,virtual,registration_url,start_date,end_date,program,category,audience,...,zipcode,contact_name,contact_phone,contact_email,meeting_type,price,start_time,end_time,internal_id,audience_max
0,Grab and Go: Paper Rose Craft,In celebration of Latinx History Month come&nb...,https://chipublib.bibliocommons.com/events/615...,false,https://chipublib.bibliocommons.com/events/615...,2021-10-08,2021-10-08,Latinx History,232,Kids,...,60612,Ms. Jade,3127466800,jnewman@chipublib.org,face_to_face,0,09:00,09:00,61571f6f66359037007e6f8f,Kids
1,Rise and Shine Story Time: The Alphabet,Start your day the library way!&nbsp;Join Mr. ...,https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2021-10-08,2021-10-08,null,230,"Babies, Preschoolers, Toddlers",...,null,Jerry McNally,3127441965,null,online,0,09:30,09:30,615b087bddad1a3000c2254f,"Babies, Preschoolers, Toddlers"
2,Family Story Time,Join the Thomas Hughes Children's Library staf...,https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-08,2021-10-08,null,230,"Babies, Preschoolers, Toddlers",...,null,Thomas Hughes Children's Library,3127474200,null,online,0,10:00,10:00,612e35a6577350360031868f,"Babies, Preschoolers, Toddlers"
3,Creative Writing Workshop,"Turn over a new leaf, this autumn, and celebra...",https://chipublib.bibliocommons.com/events/612...,true,https://chipublib.bibliocommons.com/events/612...,2021-10-08,2021-10-08,null,230,Adults,...,null,Tom Stark,3127429590,tstark@chipublib.org,online,0,10:00,10:00,612fc678cf94553600290f05,Adults
4,Story Time with Ms. Katie,Join Ms.&nbsp;Katie from the McKinley Park Bra...,https://chipublib.bibliocommons.com/events/610...,true,https://chipublib.bibliocommons.com/events/610...,2021-10-08,2021-10-08,null,230,"Babies, Preschoolers, Toddlers",...,null,Ms. Katie,3127476082,kboucher@chipublib.org,online,0,10:30,10:30,610d3daf8b28013800490fd8,"Babies, Preschoolers, Toddlers"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292,Books 'n Babes,"Come one, come all, to our fun story time for ...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-02-28,2022-02-28,null,230,Babies,...,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,0,10:30,10:30,61520689d68b0a2f003c329,Babies
1293,Books 'n Babes,"Come one, come all, to our fun story time for ...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-03-14,2022-03-14,null,230,Babies,...,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,0,10:30,10:30,61520689d68b0a2f003c329d,Babies
1294,Books 'n Babes,"Come one, come all, to our fun story time for ...",https://chipublib.bibliocommons.com/events/615...,true,https://chipublib.bibliocommons.com/events/615...,2022-03-28,2022-03-28,null,230,Babies,...,null,Jenny Hargrave,3127472733,jhargrave@chipublib.org,online,0,10:30,10:30,61520689d68b0a2f003c329,Babies
1295,"Law at the Library: Wills, Trusts and Estate P...",Each Law at the Library program features a pre...,https://chipublib.bibliocommons.com/events/611...,true,https://chipublib.bibliocommons.com/events/611...,2022-03-29,2022-03-29,Law at the Library,224,Adults,...,null,adultservices@chipublib.org,null,adultservices@chipublib.org,online,0,18:00,18:00,611ab2eaad0dd42800ed2751,Adults


In [27]:
df.to_csv('test_run.csv')

In [28]:
aud_list=['Babies','Toddlers','Preschoolers','Kids','Tweens','Teens','Adults','Seniors','All Ages']

In [29]:
babymin,babymax=0,2
babyrange=list(range(babymin, babymax+1))
toddlermin,toddlermax=1,3
toddlerrange=list(range(toddlermin, toddlermax+1))
prekmin,prekmax=2,4
prekrange=list(range(prekmin, prekmax+1))
kidmin,kidmax=4,10
kidrange=list(range(kidmin, kidmax+1))
tweenmin,tweenmax=9,12
tweenrange=list(range(tweenmin, tweenmax+1))
teenmin,teenmax=13,18
teenrange=list(range(teenmin, teenmax+1))
adultmin,adultmax=18,99
adultrange=list(range(adultmin, adultmax+1))
seniormin,seniormax=65,99
seniorrange=list(range(seniormin, seniormax+1))
allmin,allmax=0,99
allrange=list(range(allmin, allmax+1))

In [30]:
aud_min=[babymin,toddlermin,prekmin,kidmin,tweenmin,teenmin,adultmin,seniormin,allmin]
aud_min=list(map(str, aud_min))
aud_max=[babymax,toddlermax,prekmax,kidmax,tweenmax,teenmax,adultmax,seniormax,allmax]
aud_max=list(map(str, aud_max))

In [31]:
audhelpmin = {'aud_list':aud_list,
      'aud_min':aud_min}

audhelpmin = dict(zip(audhelpmin['aud_list'], audhelpmin['aud_min']))

In [32]:
for old, new in audhelpmin.items():
    df['audience'] = df['audience'].str.replace(old, new, regex=False)

In [33]:
df['audience'] = df['audience'].str.replace(' ','')

In [34]:
minlist=[]
for x in df['audience'].str.split(','):
    minlist.append(min(x))
    
df['min_age']=minlist

In [35]:
audhelpmax = {'aud_list':aud_list,
      'aud_max':aud_max}

audhelpmax = dict(zip(audhelpmax['aud_list'], audhelpmax['aud_max']))

In [36]:
for old, new in audhelpmax.items():
    df['audience_max'] = df['audience_max'].str.replace(old, new, regex=False)

In [37]:
df['audience_max'] = df['audience_max'].str.replace(' ','')

In [38]:
maxlist=[]
for x in df['audience_max'].str.split(','):
    maxlist.append(max(x))
    
df['max_age']=maxlist

In [39]:
df.to_csv('test_run.csv')

In [40]:
records=df.to_dict('records')

In [41]:
from pyairtable import Api, Base, Table